# 小学期·爬虫笔记（二）

### 分析网页

豆瓣提供了标签功能用以搜索电影，如搜索电视剧便可以使用tag [“电视剧”](https://movie.douban.com/tag/#/?sort=S&range=0,10&tags=%E7%94%B5%E8%A7%86%E5%89%A7)

<img src="image\2-1.PNG" width="800" hegiht="300" align=center />

如上界面，可以看到，除了tag之外，还有评分区间等诸多筛选条件，每个条目有图片、名称及评分。由于每一页显示的范围有限，因此需要我们点击下一页来加载更多的内容，可以考虑打开开发者工具来观察点击加载时调用资源的变化，来找到查找的接口

![开发者工具-netsource](image\2-2.png)

可以看到，网页发送了一个名为“ https://movie.douban.com/j/new_search_subjects?sort=S&range=0,10&tags=%E7%94%B5%E8%A7%86%E5%89%A7&start=0” 的请求连接，再往下看它的请求参数 ![请求参数](image\2-3.png)

这里面包含了一些参数：
* sort是排序方式，s代表按评分排序，同样的还有时间R和热度T
* range是分数范围，0,10表示评分区间为0-10的电影，同理0，5表示评分为5分以下的电影
* tags就是标签了，想要多个标签就在后面加逗号，如 “tags=电视剧,励志”，就表示标签为电视剧和励志
* start指起点，0就表示从头开始搜索，显示20个，20就表示从20开始，搜到39

我们试下，把参数输入到链接中：https://movie.douban.com/j/new_search_subjects?sort=S&range=0,10&tags=%E7%94%B5%E8%A7%86%E5%89%A7&start=10 得到了如下的东西：![](image\2-4.png)

这个，便是用JSON存储的信息。可以看到，里面包含了：
* directors：导演
* rate：评分
* title：标题
* url：电影链接
* casts：主演
* cover：海报
* id：编号

可以考虑编写代码来构造请求，读取上面的信息

In [19]:
import requests

In [4]:
# 请求参数
qs = {"sort":["s","r","t"],
      "range":["0","1","2","3","4","5","6","7","8","9","10"],
      "tags":[],
      "start":["0"]}

In [12]:
# 利用参数构建url
url = " https://movie.douban.com/j/new_search_subjects?sort=%s&range=%s,%s&tags=%%E7%%94%%B5%%E8%%A7%%86%%E5%%89%%A7&start=%s" % (qs["sort"][0], qs["range"][0],qs["range"][10],qs["start"][0])

In [13]:
r = requests.get(url)

In [27]:
# 获取json数据
result = r.json()

In [28]:
result["data"][0]

{'casts': ['戴维·阿滕伯勒'],
 'cover': 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2518413249.jpg',
 'cover_x': 1530,
 'cover_y': 2176,
 'directors': ['詹姆斯·霍尼伯内'],
 'id': '26979545',
 'rate': '9.9',
 'star': '50',
 'title': '蓝色星球2',
 'url': 'https://movie.douban.com/subject/26979545/'}

可以看到，这个就是排名第一的电视剧的详细内容，可以考虑将其写入表格或文本中，这样，我们便得到了爬取豆瓣电视剧的方法，可以根据结果中电视剧的URL来进一步提取电视剧的相关内容